In [2]:
import multiprocessing
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
cores = multiprocessing.cpu_count()
print("cpu number is :", cores)


# 第一步 数据预处理，
def cut(document):
    # 定义删除除字母,数字，汉字以外的所有符号的函数
    doc = nltk.word_tokenize(document)
    filtered = [w for w in doc if (w not in stopwords.words('english') and w not in '!@#$%^&*()_+<>?":,./;{}[]')]
    return filtered


df_train = pd.read_csv('/Users/yuchk/PycharmProjects/IMDB/0_dataset/orign/train_imdb.tsv',
                 usecols=['tag', 'sen'], sep='\t')
df_test = pd.read_csv('/Users/yuchk/PycharmProjects/IMDB/0_dataset/orign/test_imdb.tsv',
                 usecols=['tag', 'sen'], sep='\t')
# df['cat_id'] = df['product_type'].factorize()[0]
# cat_id_df = df[['product_type', 'cat_id']].drop_duplicates().sort_values('cat_id').reset_index(drop=True)
# cat_to_id = dict(cat_id_df.values)
# id_to_cat = dict(cat_id_df[['cat_id', 'product_type']].values)
# d = {'cat': df['product_type'].value_counts().index, 'count': df['product_type'].value_counts()}
# df_cat = pd.DataFrame(data=d)
# # my_cont_df = df_cat[df_cat['count'] > 5000]['cat']
# # df = df[df['product_type'].isin(list(my_cont_df))]
# my_min_df = df_cat[df_cat['count'] > 3000]['cat']
# my_max_df = df_cat[df_cat['count'] < 6000]['cat']
# df = df[df['product_type'].isin(list(my_min_df))]
# df = df[df['product_type'].isin(list(my_max_df))]
# # 分词，并过滤停用词
# # df['cut_title'] = df['title'].apply(cut)

print(df_train)
print(df_test)

cpu number is : 4
       tag                                                sen
0        0  It really boggles my mind when someone comes a...
1        0  Mary Pickford becomes the chieftain of a Scott...
2        0  Well, at least my theater group did, lol. So o...
3        1  I must give How She Move a near-perfect rating...
4        0  I must say, when I read the storyline on the b...
...    ...                                                ...
19995    1  Simple, meaningful and delivers an emotional p...
19996    1  I'm fan of ART, I like anything about Art, I l...
19997    0  Despite being a sequel to the more potent orig...
19998    0  Also known in a different form as "House of Ex...
19999    0  This has the absolute worst performance from R...

[20000 rows x 2 columns]
      tag                                                sen
0       1  The first one meant victory. This one means de...
1       1  Excellent movie, a realistic picture of contem...
2       1  This film is movin

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_train['sen'], df_train['tag'], random_state=42, test_size = 0.2,stratify=df['tag'])



In [6]:
tf = TfidfVectorizer()
X_train_tfidf = tf.fit_transform(X_train)
X_test_tfidf = tf.transform(X_test)


In [7]:

kn = KNeighborsClassifier(n_jobs=2,n_neighbors=1).fit(X_train_tfidf, y_train)
y_pre_kn = kn.predict(X_test_tfidf)

print('Testing accuracy %s' % accuracy_score(y_test, y_pre_kn))

Testing accuracy 0.751


In [8]:

kn = KNeighborsClassifier(n_jobs=2,n_neighbors=2).fit(X_train_tfidf, y_train)
y_pre_kn = kn.predict(X_test_tfidf)

print('Testing accuracy %s' % accuracy_score(y_test, y_pre_kn))

Testing accuracy 0.7435


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pre_kn))

In [10]:
rfc = RandomForestClassifier(n_estimators=200, max_depth=4, random_state=0).fit(X_train_tfidf, y_train)
y_pre_rfc = rfc.predict(X_test_tfidf)

print('Testing accuracy %s' % accuracy_score(y_test, y_pre_rfc))

Testing accuracy 0.80825


In [11]:
ls = LinearSVC().fit(X_train_tfidf, y_train)
y_pre_ls = ls.predict(X_test_tfidf)

print('Testing accuracy %s' % accuracy_score(y_test, y_pre_ls))

Testing accuracy 0.89825


In [12]:
lr = LogisticRegression(random_state=0).fit(X_train_tfidf, y_train)
y_pre_lr = lr.predict(X_test_tfidf)

print('Testing accuracy %s' % accuracy_score(y_test, y_pre_lr))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Testing accuracy 0.88775


In [13]:
from  sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(192,)).fit(X_train_tfidf, y_train)
y_pre_mlp = mlp.predict(X_test_tfidf)
m1 = accuracy_score(y_test, y_pre_mlp)
print('Testing accuracy %s' % m1)


Testing accuracy 0.899


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [14]:
from sklearn.linear_model import SGDClassifier
sgdc = SGDClassifier()
sgdc.fit(X_train_tfidf, y_train)
sgdc_predict_y = sgdc.predict(X_test_tfidf)
sgdr = accuracy_score(y_test, sgdc_predict_y)
print(sgdr)

0.896


In [17]:
df['cat_id'] = df['tag'].factorize()[0]
cat_id_df = df[['tag', 'cat_id']].drop_duplicates().sort_values('cat_id').reset_index(drop=True)
cat_to_id = dict(cat_id_df.values)
id_to_cat = dict(cat_id_df[['cat_id', 'tag']].values)
d = {'cat': df['tag'].value_counts().index, 'count': df['tag'].value_counts()}
df_cat = pd.DataFrame(data=d)
# my_cont_df = df_cat[df_cat['count'] > 5000]['cat']
# df = df[df['product_type'].isin(list(my_cont_df))]

# 分词，并过滤停用词
# df['cut_title'] = df['title'].apply(cut)